In [2]:
from kamene.all import *
from pynq_networking import *
from math import log2 

A = 0.1 # Smoothing factor
WINDOW_SIZE = 30 # in seconds
OVERLAP = 0.8
INTERVAL_SEC = WINDOW_SIZE - WINDOW_SIZE*OVERLAP
PARAMS = ["src", "sport", "dport"]

# Observed entropies
H = []

# Predicted entropies
H_predict = []

# Parameters entropy prediction error
err = []

# Anomaly score
d = 0

In [ ]:
overlap_flows = [[] for i in range(OVERLAP/(1-OVERLAP))]
overlap_flows_count = [[] for i in range(OVERLAP/(1-OVERLAP))]
overlap_flows_pkts = [0 for i in range(OVERLAP/(1-OVERLAP))]
current_flows = []
current_flows_count = []
current_packets = 0
def shift_flows():
    for i in range(len(overlap_flows-1)):
        overlap_flows[i] = overlap_flows[i+1]
        overlap_flows_count[i] = overlap_flows_count[i+1]
        overlap_flows_pkts[i] = overlap_flows_pkts[i+1]
    overlap_flows[len(overlap_flows)-1] = current_flows
    overlap_flows[len(overlap_flows)-1] = current_flows_count
    overlap_flows_pkts[len(overlap_flows)-1] = current_packets

In [1]:
def packet_processing(packet):
    current_packets =  current_packets + 1
    flow = {}
    if(TCP in packet):
        flow = {"src": packet[TCP].src, "sport":packet[TCP].sport, "dport": packet[TCP].dport}            
    elif(UDP in packet):
        flow = {"src": packet[UDP].src, "sport":packet[UDP].sport, "dport": packet[UDP].dport}
    if(flow != {} and (flow not in current_flows):
        current_flows.append(flow)
        current_flows_count.append(1)
    else:
        i = current_flows.index(flow)
        current_flows_count[i] = current_flows_count[i] + 1

In [ ]:
def calc_entropy(param):
    values = []
    freqs = []
    for i in len(overlap_flows):
        for j in len(overlap_flows[i]):
            if((overlap_flows[i][j][param] not in values):
                values.append(overlap_flows[i][j][param])
                freqs.append(1)
            else:
                index = values.index(overlap_flows[i][j][param])
                freqs[index] = freq[index] + overlap_flows_count[i][j]
    entropy = 0
    for i in range(len(freq)):
        p = freqs[i][param] / current_packets
        entropy = entropy - p*log2(p)
    entropy = entropy / log2(current_packets) # Normalize entropy
    return entropy
    
def predict_entropy(param):
    return H_predict[PARAMS.index(param)]*(1-A) + H[PARAMS.index(param)]*A
    
def calc_error(params):
    return abs(H_predict[PARAMS.index(param)]-H[PARAMS.index(param)])

In [ ]:
for i in range(len(overlap_flows)):
    clear_output(wait=True)
    pkt_trace = sniff(iface="br0",
                      timeout = INTERVAL_SEC, filter="tcp or udp",
                      prn=lambda x: packet_processing(x))
    shift_flows()
    
pkt_trace = sniff(iface="br0",
                  timeout = INTERVAL_SEC, filter="tcp or udp",
                  prn=lambda x: packet_processing(x))
H = map(PARAMS, calc_entropy)
H_predict = map(PARAMS, predict_entropy)

In [ ]:
while(True):
    clear_output(wait=True)
    pkt_trace = sniff(iface="br0",
                      timeout = INTERVAL_SEC, filter="tcp or udp",
                      prn=lambda x: packet_processing(x))
    H = map(PARAMS, calc_entropy)
    err = map(PARAMS, calc_error)
    draw_graphs()
    H_predict = map(PARAMS, predict_entropy)

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

def draw_graphs():
    while(True):
        types = ["ARP" if ARP in x else 
                 "MQTTSN" if MQTTSN in x else
                 "IPV6_UDP" if ((UDP in x) and (IPv6 in x)) else
                 "UDP" if (UDP in x) else
                 "IPV6_TCP" if ((TCP in x) and (IPv6 in x)) else
                 "TCP" if (TCP in x) else
                 "Unknown" for x in pkt_trace]
        counts = Counter(types)
        frequencies = counts.values()
        names = [x for x in counts.keys()]
        centers = range(len(counts))
        plt.bar(centers, frequencies, align='center', tick_label=names)
        plt.title("Histogram of received packet types")
        plt.draw()
        plt.pause(INTERVAL_SEC)
        plt.cla()